-csv datei einlesen
-Name ändern
- 1sten 5 zeilen ausgeben

In [1]:
import pandas as pd
from datetime import datetime, timedelta
csv_file = '../Data_Cleanup/outCSV/Clean_Metro_Set.csv'
metro = pd.read_csv(csv_file)
print(metro.head())
metro.info()

C:\Users\yusuf\AppData\Local\Temp\ipykernel_70780\2191921915.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  metro = pd.read_csv(csv_file)


   Unnamed: 0.1  Unnamed: 0           start_time             end_time  \
0             0           0  2018-05-29 16:45:00  2018-05-29 17:08:00   
1             1         891  2019-08-16 17:12:00  2019-08-16 17:42:00   
2             2        1467  2019-01-30 12:22:00  2019-01-30 12:28:00   
3             3        6956  2018-05-15 08:23:00  2018-05-15 08:30:00   
4             4       11733  2018-11-03 18:30:00  2018-11-03 18:51:00   

   start_station_id  start_station_lat  start_station_lon  end_station_id  \
0              3068            34.0532        -118.250954            3014   
1              3068            34.0532        -118.250954            3014   
2              3068            34.0532        -118.250954            3014   
3              3068            34.0532        -118.250954            3014   
4              3068            34.0532        -118.250954            3014   

   end_station_lat  end_station_lon bike_id  
0         34.05661      -118.237213    5772  
1     

-Tabelle sortieren nach Transaktionszeit

In [2]:
metro['start_time'] = pd.to_datetime(metro['start_time'])

metro ['end_time'] = pd.to_datetime(metro['end_time'])

metro = metro.sort_values(by='end_time')

-Neue Spalte hinzufügen für Idle_Time
-Alle neuen felder mit 0 initialisieren


In [9]:
metro['idle_time'] = pd.Timedelta(0)

In [10]:
metro.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1552782 entries, 690312 to 373108
Data columns (total 12 columns):
 #   Column             Non-Null Count    Dtype          
---  ------             --------------    -----          
 0   Unnamed: 0.1       1552782 non-null  int64          
 1   Unnamed: 0         1552782 non-null  int64          
 2   start_time         1552782 non-null  datetime64[ns] 
 3   end_time           1552782 non-null  datetime64[ns] 
 4   start_station_id   1552782 non-null  int64          
 5   start_station_lat  1552782 non-null  float64        
 6   start_station_lon  1552782 non-null  float64        
 7   end_station_id     1552782 non-null  int64          
 8   end_station_lat    1552782 non-null  float64        
 9   end_station_lon    1552782 non-null  float64        
 10  bike_id            1552782 non-null  object         
 11  idle_time          1552782 non-null  timedelta64[ns]
dtypes: datetime64[ns](2), float64(4), int64(4), object(1), timedelta64

Idle Time berechnung

Erklärung des folgenden Codes:

Hier beginnt eine Schleife, die jede Zeile in den Daten durchläuft.

Die aktuelle Station ID wird aus der Spalte 'start_station_id' der aktuellen Zeile abgerufen und in der Variable `current_station_id` gespeichert.

Daraufhin folgt die Filterung aller vorherigen Transaktionen, bei denen die 'end_station_id' mit der 'current_station_id' übereinstimmt und die Zeilenindizes kleiner als der aktuelle Index `i` sind.

Der darauf Folgende Block wird nur ausgeführt, wenn es vorherige Transaktionen an derselben Station gibt.

Die letzte Transaktion mit der passenden 'end_station_id'wird ausgewählt, um die endgültige Zeit für die Idle Time-Berechnung zu erhalten.

Die Variable `count` wird initialisiert, um die Idle Time zu zählen, und `temp` wird als temporäre Variable für die Zeitberechnung verwendet.

In dieser Schleife wird die Zeit erhöht, indem 'count' und 'temp' um 1 Minute inkrementiert werden, bis `temp` der 'start_time' der aktuellen Transaktion entspricht.

Die Idle Time für die aktuelle Zeile wird auf den berechneten Wert `count` gesetzt.




In [ ]:
#Test
import pandas as pd
a = pd.Series(dtype='datetime64[ns]')
b = pd.to_datetime('2023-05-15')
d = pd.to_datetime('2023-05-17')
c = pd.to_datetime('2023-05-16')
a.loc[len(a)] = b
a.loc[len(a)] = d
filtered_series = a[a < c]
time = c - filtered_series.max()
time

In [6]:
BigDict = {}
for i, row in metro.iterrows():
    current_start_station_id = metro.loc[i, "start_station_id"]
    current_end_station_id = metro.loc[i, "end_station_id"]
    current_start_time = metro.loc[i, "start_time"]
    current_end_time = metro.loc[i, "end_time"]
    try:
        BigDict[str(current_end_station_id)].loc[len(str(current_end_station_id))] = current_end_time
    except KeyError:
        BigDict[str(current_end_station_id)] = pd.Series(dtype='datetime64[ns]')
        BigDict[str(current_end_station_id)].loc[len(str(current_end_station_id))] = current_end_time


    try:
        filtered_series = BigDict[str(current_start_station_id)][BigDict[str(current_start_station_id)] < current_start_time]
        metro.at[i, 'idle_time'] = current_start_time - filtered_series.max()
    except KeyError:
        BigDict[str(current_end_station_id)] = pd.Series(dtype='datetime64[ns]')

In [ ]:
BigDict = {}
for i, row in metro.iterrows():
    current_start_station_id = metro.loc[i, "start_station_id"]
    current_end_station_id = metro.loc[i, "end_station_id"]
    current_start_time = metro.loc[i, "start_time"]
    current_end_time = metro.loc[i, "end_time"]
    try:
        metro.at[i, "idle_time"] = current_start_time - BigDict[str(current_start_station_id)]
    except KeyError:
        BigDict[str(current_end_station_id)] = current_end_time
    BigDict[str(current_end_station_id)] = current_end_time

In [7]:
metro.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1552782 entries, 690312 to 373108
Data columns (total 12 columns):
 #   Column             Non-Null Count    Dtype          
---  ------             --------------    -----          
 0   Unnamed: 0.1       1552782 non-null  int64          
 1   Unnamed: 0         1552782 non-null  int64          
 2   start_time         1552782 non-null  datetime64[ns] 
 3   end_time           1552782 non-null  datetime64[ns] 
 4   start_station_id   1552782 non-null  int64          
 5   start_station_lat  1552782 non-null  float64        
 6   start_station_lon  1552782 non-null  float64        
 7   end_station_id     1552782 non-null  int64          
 8   end_station_lat    1552782 non-null  float64        
 9   end_station_lon    1552782 non-null  float64        
 10  bike_id            1552782 non-null  object         
 11  idle_time          1157431 non-null  timedelta64[ns]
dtypes: datetime64[ns](2), float64(4), int64(4), object(1), timedelta64

In [8]:
metro.to_csv('../Data_Cleanup/outCSV/IdleTimeTest2.csv')